## Basic setup & paths

In [11]:
import os
import sys
import random
import torch
import pandas as pd

PROJECT_ROOT = "/root/autodl-tmp/csc521-project"
CODE_ROOT = os.path.join(PROJECT_ROOT, "code")
DATA_DIR = os.path.join(CODE_ROOT, "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "evaluation_results")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# so that `from models...` etc works if you later want to import modules
sys.path.append(CODE_ROOT)

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
print("PROJECT_ROOT:", PROJECT_ROOT)


Using device: cuda
PROJECT_ROOT: /root/autodl-tmp/csc521-project


In [12]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## SDXL loader

In [13]:
from diffusers import StableDiffusionXLPipeline

def load_sdxl(device: str = "cuda"):
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    
    print(f"Loading SDXL from {model_id} on {device} (fp32)...")

    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float32,     # use fp32 for stable grads
        use_safetensors=True,
    ).to(device)

    return pipe

## CLIP loader

In [14]:
from transformers import CLIPProcessor, CLIPModel

def load_clip(device: str = "cuda"):
    model_id = "openai/clip-vit-base-patch32"
    
    print(f"Loading CLIP model: {model_id}...")
    
    try:
        model = CLIPModel.from_pretrained(
            model_id, 
            use_safetensors=True
        ).to(device)
        processor = CLIPProcessor.from_pretrained(model_id)
    except Exception as e:
        print(f"Error loading CLIP with safetensors: {e}")
        print("Trying fallback...")
        model = CLIPModel.from_pretrained(model_id).to(device)
        processor = CLIPProcessor.from_pretrained(model_id)

    return model, processor

## Load models

In [15]:
print("Loading SDXL & CLIP...")
pipe = load_sdxl(device)
clip_model, clip_processor = load_clip(device)

Loading SDXL & CLIP...
Loading SDXL from stabilityai/stable-diffusion-xl-base-1.0 on cuda (fp32)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading CLIP model: openai/clip-vit-base-patch32...


## Generate prompt–target CSV

5 sample 

In [16]:
import os
import json
import random
import pandas as pd

if 'DATA_DIR' not in locals():
    DATA_DIR = "/root/autodl-tmp/csc521-project/code/data"
    os.makedirs(DATA_DIR, exist_ok=True)

EXPERIMENT_CSV = os.path.join(PROJECT_ROOT, "experiment_log.csv")
print("Experiment CSV path:", EXPERIMENT_CSV)

LOCAL_COCO_JSON = "/root/autodl-tmp/csc521-project/datasets/coco2017/annotations/captions_val2017.json"



with open(LOCAL_COCO_JSON, "r") as f:
    coco_data = json.load(f)

all_captions = [item['caption'] for item in coco_data['annotations']]
print(f"Total captions: {len(all_captions)}")

num_samples = 100
prompt_target_pairs = []
selected_indices = random.sample(range(len(all_captions)), num_samples)

for idx in selected_indices:
    p = all_captions[idx]
    t = random.choice(all_captions)
    while t == p:
        t = random.choice(all_captions)
    prompt_target_pairs.append({"prompt_p": p, "target_t": t})

df_pairs = pd.DataFrame(prompt_target_pairs)
df_pairs.to_csv(EXPERIMENT_CSV, index=False)
print("Saved pairs to:", EXPERIMENT_CSV)
print(df_pairs.head())

Experiment CSV path: /root/autodl-tmp/csc521-project/experiment_log.csv
Total captions: 5003
Saved pairs to: /root/autodl-tmp/csc521-project/experiment_log.csv
                                            prompt_p  \
0  a woman standing on a sidewalk while using a c...   
1  Cat perched on the sofa looking out of the window   
2  A man with his shirt off sitting by the shorel...   
3  a happy dog playing with a frisbee by a mossy ...   
4  A road sign on top of a stop sign at an inters...   

                                            target_t  
0  A rode that has car and bicycle lane that cars...  
1  Stuffed toy bear sitting on dashboard of motor...  
2  A train station with an awning is depicted wit...  
3  three birds perched and one flying towards the...  
4      Two people with heavy winter coats are eating  


entire dataset

In [17]:
# import os
# import json
# import pandas as pd

# LOCAL_COCO_JSON = "/root/autodl-tmp/csc521-project/datasets/coco2017/annotations/captions_val2017.json"
# EXPERIMENT_CSV = os.path.join(PROJECT_ROOT, "experiment_log_full.csv")

# print("Loading COCO captions...")
# with open(LOCAL_COCO_JSON, "r") as f:
#     coco_data = json.load(f)

# all_captions = [item['caption'] for item in coco_data['annotations']]
# print(f"Total COCO captions: {len(all_captions)}")

# # Create a clean-target pairing for EVERY caption
# prompt_target_pairs = []
# for p in all_captions:
#     t = random.choice(42)
#     while t == p:
#         t = random.choice(42)
#     prompt_target_pairs.append({"prompt_p": p, "target_t": t})

# df_pairs = pd.DataFrame(prompt_target_pairs)
# df_pairs.to_csv(EXPERIMENT_CSV, index=False)

# print("Saved full dataset pairs to:", EXPERIMENT_CSV)
# df_pairs.head()


## Inspect the CSV

In [18]:
df = pd.read_csv(EXPERIMENT_CSV)
print(df.shape)
df.head()


(100, 2)


,prompt_p,target_t
0,a woman standing on a sidewalk while using a c...,A rode that has car and bicycle lane that cars...
1,Cat perched on the sofa looking out of the window,Stuffed toy bear sitting on dashboard of motor...
2,A man with his shirt off sitting by the shorel...,A train station with an awning is depicted wit...
3,a happy dog playing with a frisbee by a mossy ...,three birds perched and one flying towards the...
4,A road sign on top of a stop sign at an inters...,Two people with heavy winter coats are eating


# White-box SDXL attacker

In [19]:
import torch
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, Dict, Any
from diffusers import DDIMScheduler
import torchvision.transforms as transforms


# ============================================================
# AttackConfig
# ============================================================
@dataclass
class AttackConfig:
    num_diffusion_steps: int = 20
    num_attack_iters: int = 5
    guidance_scale: float = 0.0
    lr: float = 1e-2             # unused now (manual PGD)
    lambda_orig: float = 0.5
    latent_res: int = 64
    verbose: bool = True

    # PGD parameters
    eps_l2: float = 10.0
    step_size: float = 1.0



# ============================================================
# WhiteBoxSDXLAttacker
# ============================================================
class WhiteBoxSDXLAttacker:
    def __init__(self, pipe, clip_model, clip_processor, device="cuda"):
        self.pipe = pipe
        self.device = device
        self.clip_model = clip_model
        self.clip_processor = clip_processor

        # replace scheduler with DDIM
        self.pipe.scheduler = DDIMScheduler.from_config(self.pipe.scheduler.config)
        self.scheduler = self.pipe.scheduler

        # disable VAE tiling/slicing
        self.pipe.disable_vae_slicing()
        self.pipe.disable_vae_tiling()

        # unpack components
        self.unet = pipe.unet
        self.vae = pipe.vae
        self.text_encoder = pipe.text_encoder
        self.text_encoder_2 = pipe.text_encoder_2
        self.tokenizer = pipe.tokenizer
        self.tokenizer_2 = pipe.tokenizer_2

        # send everything to device (fp32 for gradients)
        self.pipe.to(self.device)
        self.unet.to(self.device, dtype=torch.float32)
        self.vae.to(self.device, dtype=torch.float32)
        self.text_encoder.to(self.device, dtype=torch.float32)
        self.text_encoder_2.to(self.device, dtype=torch.float32)
        self.clip_model.to(self.device, dtype=torch.float32)

        # freeze weights
        self.unet.eval().requires_grad_(False)
        self.vae.eval().requires_grad_(False)
        self.clip_model.eval().requires_grad_(False)
        self.text_encoder.eval().requires_grad_(False)
        self.text_encoder_2.eval().requires_grad_(False)

        # allow gradients for UNet inputs (needed for PGD)
        self.unet.enable_gradient_checkpointing()
        def hook(_, __, output):
            output.requires_grad_(True)
        self.unet.conv_in.register_forward_hook(hook)

        # CLIP normalization buffers
        self.clip_mean = torch.tensor(
            [0.48145466, 0.4578275, 0.40821073],
            device=self.device
        ).view(1,3,1,1)
        self.clip_std = torch.tensor(
            [0.26862954, 0.26130258, 0.27577711],
            device=self.device
        ).view(1,3,1,1)



    # ======================================================
    # Manual SDXL prompt encoding (keeps gradient clean)
    # ======================================================
    def _encode_prompt_manual(self, prompt: str):
        # encoder 1
        ti = self.tokenizer(prompt, padding="max_length",
                            max_length=self.tokenizer.model_max_length,
                            truncation=True, return_tensors="pt")
        ids = ti.input_ids.to(self.device)

        # encoder 2
        ti2 = self.tokenizer_2(prompt, padding="max_length",
                               max_length=self.tokenizer_2.model_max_length,
                               truncation=True, return_tensors="pt")
        ids2 = ti2.input_ids.to(self.device)

        enc1 = self.text_encoder(ids, output_hidden_states=True)
        enc2 = self.text_encoder_2(ids2, output_hidden_states=True)

        hidden1 = enc1.hidden_states[-2]
        hidden2 = enc2.hidden_states[-2]

        prompt_embeds = torch.cat([hidden1, hidden2], dim=-1)
        pooled = enc2[0]
        return prompt_embeds.float(), pooled.float()



    # ======================================================
    # CLIP preprocessing + scoring
    # ======================================================
    def _differentiable_clip_preprocess(self, images: torch.Tensor):
        images = F.interpolate(images, (224,224), mode="bicubic", align_corners=False)
        return (images - self.clip_mean) / self.clip_std

    def _clip_scores(self, images, clean_prompt, target_prompt):
        img_inputs = self._differentiable_clip_preprocess(images)
        txt = self.clip_processor(text=[target_prompt, clean_prompt],
                                  return_tensors="pt",
                                  padding=True, truncation=True).to(self.device)

        out = self.clip_model(
            pixel_values=img_inputs,
            input_ids=txt.input_ids,
            attention_mask=txt.attention_mask,
        )

        img_emb = F.normalize(out.image_embeds, dim=-1)
        txt_emb = F.normalize(out.text_embeds, dim=-1)

        sim_target = (img_emb * txt_emb[0:1]).sum(-1).item()
        sim_orig   = (img_emb * txt_emb[1:2]).sum(-1).item()
        return {"sim_target": sim_target, "sim_orig": sim_orig}



    # ======================================================
    # Single prompt -> image via DDIM (same as attack loop)
    # ======================================================
    def _generate_image_from_embeds(self, prompt_embeds, pooled_embeds,
                                    num_steps, guidance_scale, seed, latent_res):

        bsz = 1
        dtype = torch.float32

        latents = torch.randn(
            (bsz, self.unet.config.in_channels, latent_res, latent_res),
            generator=torch.Generator(device=self.device).manual_seed(seed),
            device=self.device, dtype=dtype,
        )
        latents *= self.scheduler.init_noise_sigma

        self.scheduler.set_timesteps(num_steps, device=self.device)
        timesteps = self.scheduler.timesteps

        add_time_ids = torch.tensor(
            [[1024,1024,0,0,1024,1024]],
            device=self.device, dtype=dtype
        )

        added_cond_kwargs = {
            "text_embeds": pooled_embeds,
            "time_ids": add_time_ids,
        }

        # simple DDIM loop (no CFG)
        for t in timesteps:
            latent_in = self.scheduler.scale_model_input(latents, t)
            noise = self.unet(
                latent_in, t,
                encoder_hidden_states=prompt_embeds,
                added_cond_kwargs=added_cond_kwargs,
            ).sample
            latents = self.scheduler.step(noise, t, latents).prev_sample

        # decode
        latents = latents / self.vae.config.scaling_factor
        image = self.vae.decode(latents).sample
        image = (image/2 + 0.5).clamp(0,1)
        return image



    # ======================================================
    # Main PGD attack
    # ======================================================
    def attack(self, clean_prompt, target_prompt, cfg=None):
        if cfg is None:
            cfg = AttackConfig()

        if cfg.verbose:
            print(f"[CONFIG] eps_l2={cfg.eps_l2}, step_size={cfg.step_size}")

        # encode clean prompt
        with torch.no_grad():
            clean_embeds, pooled_embeds = self._encode_prompt_manual(clean_prompt)

        adv_embeds = clean_embeds.clone().detach().requires_grad_(True)

        # clean scores
        with torch.no_grad():
            clean_img = self._generate_image_from_embeds(
                clean_embeds, pooled_embeds,
                num_steps=cfg.num_diffusion_steps,
                guidance_scale=cfg.guidance_scale,
                seed=0, latent_res=cfg.latent_res
            )
            clean_scores = self._clip_scores(clean_img, clean_prompt, target_prompt)

        if cfg.verbose:
            print(f"[CLEAN] sim_target={clean_scores['sim_target']:.3f}")

        best_obj = -1e9
        best_embeds = clean_embeds.clone()
        best_scores = clean_scores

        # --------------------- ATTACK LOOP ---------------------
        for it in range(cfg.num_attack_iters):

            if adv_embeds.grad is not None:
                adv_embeds.grad.zero_()

            img_adv = self._generate_image_from_embeds(
                adv_embeds, pooled_embeds,
                num_steps=cfg.num_diffusion_steps,
                guidance_scale=cfg.guidance_scale,
                seed=0, latent_res=cfg.latent_res
            )

            # CLIP loss
            img_inputs = self._differentiable_clip_preprocess(img_adv)
            txt = self.clip_processor(
                text=[target_prompt, clean_prompt],
                return_tensors="pt", padding=True, truncation=True
            ).to(self.device)

            out = self.clip_model(pixel_values=img_inputs,
                                  input_ids=txt.input_ids,
                                  attention_mask=txt.attention_mask)

            img_emb = F.normalize(out.image_embeds, dim=-1)
            txt_emb = F.normalize(out.text_embeds, dim=-1)

            sim_target = (img_emb * txt_emb[0:1]).sum(-1)
            sim_orig   = (img_emb * txt_emb[1:2]).sum(-1)
            objective  = sim_target - cfg.lambda_orig * sim_orig
            loss       = -objective

            loss.backward()

            # PGD update
            with torch.no_grad():
                grad = adv_embeds.grad
                grad_norm = grad.norm()

                if grad_norm == 0 or torch.isnan(grad_norm):
                    print(f"[Warn] Iter {it}: grad_norm invalid → skip")
                else:
                    step = cfg.step_size * grad / (grad_norm + 1e-8)
                    adv_embeds = adv_embeds - step  # descent on loss

                    # project to L2 ball
                    delta = adv_embeds - clean_embeds
                    delta_norm = delta.norm()
                    if delta_norm > cfg.eps_l2:
                        delta = delta * (cfg.eps_l2 / (delta_norm + 1e-8))

                    adv_embeds = (clean_embeds + delta).detach().requires_grad_(True)

                # save best
                if objective.item() > best_obj:
                    best_obj = objective.item()
                    best_embeds = adv_embeds.detach().clone()
                    best_scores = {
                        "sim_target": sim_target.item(),
                        "sim_orig":   sim_orig.item()
                    }

                if cfg.verbose:
                    print(
                        f"[Iter {it}] "
                        f"sim_t={sim_target.item():.4f}, "
                        f"sim_o={sim_orig.item():.4f}, "
                        f"obj={objective.item():.4f}, "
                        f"||delta||={delta_norm.item():.4f}"
                    )

        # --------------------- FINAL IMAGE ---------------------
        with torch.no_grad():
            clean_img_compare = self._generate_image_from_embeds(
                clean_embeds, pooled_embeds,
                num_steps=cfg.num_diffusion_steps,
                guidance_scale=cfg.guidance_scale,
                seed=0, latent_res=cfg.latent_res
            )
            adv_img_compare = self._generate_image_from_embeds(
                best_embeds, pooled_embeds,
                num_steps=cfg.num_diffusion_steps,
                guidance_scale=cfg.guidance_scale,
                seed=0, latent_res=cfg.latent_res
            )

            img_diff_max = (adv_img_compare - clean_img_compare).abs().max().item()
            img_diff_mean = (adv_img_compare - clean_img_compare).abs().mean().item()
            print(f"[DEBUG] Image diff: max={img_diff_max:.6f}, mean={img_diff_mean:.6f}")

            final_image = transforms.ToPILImage()(adv_img_compare[0].cpu())

        return {
            "final_image": final_image,
            "clean_clip_scores": clean_scores,
            "final_clip_scores": best_scores,
        }


## Sanity check SDXl image generation

In [20]:
# import os
# import torch
# from IPython.display import display

# # 1. Choose a safe test prompt
# prompt = "A cute dog wearing sunglasses, digital art"
# seed = 42

# print("Using device:", device)

# # 2. Generator (still fine to use CUDA even with CPU offload)
# gen = torch.Generator(device=device).manual_seed(seed)

# print("Generating image with SDXL...")
# out = pipe(
#     prompt=prompt,
#     num_inference_steps=30,
#     generator=gen,
# )

# img = out.images[0]

# # 3. Show inside notebook
# display(img)

# # 4. Save to disk under your project
# project_root = "/root/autodl-tmp/csc521-project"
# out_dir = os.path.join(project_root, "outputs")
# os.makedirs(out_dir, exist_ok=True)

# save_path = os.path.join(out_dir, "sdxl_sanity_check.png")
# img.save(save_path)

# print("Image saved to:", save_path)


## Create Attacks

In [21]:
attacker = WhiteBoxSDXLAttacker(pipe, clip_model, clip_processor, device=device)
print("Attacker ready.")

Attacker ready.


## Sanity check: run attack on a single pair

check if everything is one the same device

In [22]:
# print(next(attacker.unet.parameters()).device)
# print(next(attacker.vae.parameters()).device)
# print(next(attacker.clip_model.parameters()).device)

In [23]:
# row = df.iloc[0]
# clean_prompt = row["prompt_p"]
# target_prompt = row["target_t"]

# print("Clean prompt:", clean_prompt)
# print("Target prompt:", target_prompt)

# cfg = AttackConfig(
#     num_diffusion_steps=30,
#     num_attack_iters=20,
#     guidance_scale=0.0,
#     lambda_orig=0.5,   # start with pure target-maximization
#     latent_res=64,
#     verbose=True,
#     eps_l2=10.0,       # explicit, even though it's the default
#     step_size=5,     # explicit as well
# )

# single_result = attacker.attack(clean_prompt, target_prompt, cfg=cfg)


# print("\nClean CLIP(sim_orig):  ", single_result["clean_clip_scores"]["sim_orig"])
# print("Clean CLIP(sim_target):", single_result["clean_clip_scores"]["sim_target"])
# print("Adv   CLIP(sim_orig):  ", single_result["final_clip_scores"]["sim_orig"])
# print("Adv   CLIP(sim_target):", single_result["final_clip_scores"]["sim_target"])

# single_path = os.path.join(OUTPUT_DIR, "debug_single_adv.png")
# single_result["final_image"].save(single_path)
# print("Saved single adversarial image to:", single_path)


## Batch evaluation loop

In [24]:
results = []

cfg = AttackConfig(
    num_diffusion_steps=40,
    num_attack_iters=5,
    guidance_scale=0.0,
    lr=1e-2,
    lambda_orig=0.5,
    latent_res=64,
    verbose=True,
)

for index, row in df.iterrows():
    print(f"\n=== Index {index} ===")
    clean_prompt = row["prompt_p"]
    target_prompt = row["target_t"]

    print("Clean:", clean_prompt[:60])
    print("Target:", target_prompt[:60])

    result = attacker.attack(clean_prompt, target_prompt, cfg=cfg)

    clean_img_path = os.path.join(OUTPUT_DIR, f"{index}_clean_ref.png")
    adv_img_path = os.path.join(OUTPUT_DIR, f"{index}_adv.png")

    # reference clean image from SDXL
    clean_image = pipe(clean_prompt, num_inference_steps=30).images[0]
    clean_image.save(clean_img_path)

    if result["final_image"] is not None:
        result["final_image"].save(adv_img_path)
    else:
        print("Warning: final_image is None")

    record = {
        "index": index,
        "prompt_p": clean_prompt,
        "target_t": target_prompt,
        "clean_sim_orig": result["clean_clip_scores"]["sim_orig"],
        "clean_sim_target": result["clean_clip_scores"]["sim_target"],
        "adv_sim_orig": result["final_clip_scores"]["sim_orig"],
        "adv_sim_target": result["final_clip_scores"]["sim_target"],
        "clean_img_path": clean_img_path,
        "adv_img_path": adv_img_path,
    }
    results.append(record)

print("Done, processed:", len(results))



=== Index 0 ===
Clean: a woman standing on a sidewalk while using a cell phone 
Target: A rode that has car and bicycle lane that cars are driving o
[CONFIG] eps_l2=10.0, step_size=1.0
[CLEAN] sim_target=0.202
[Iter 0] sim_t=0.2019, sim_o=0.2074, obj=0.0982, ||delta||=1.0000
[Iter 1] sim_t=0.1942, sim_o=0.2276, obj=0.0804, ||delta||=1.0220
[Iter 2] sim_t=0.1917, sim_o=0.2114, obj=0.0860, ||delta||=1.3009
[Iter 3] sim_t=0.2005, sim_o=0.2220, obj=0.0895, ||delta||=1.3595
[Iter 4] sim_t=0.1839, sim_o=0.2037, obj=0.0820, ||delta||=1.7152
[DEBUG] Image diff: max=1.000000, mean=0.051131


  0%|          | 0/30 [00:00<?, ?it/s]


=== Index 1 ===
Clean: Cat perched on the sofa looking out of the window
Target: Stuffed toy bear sitting on dashboard of motor vehicle.
[CONFIG] eps_l2=10.0, step_size=1.0
[CLEAN] sim_target=0.220
[Iter 0] sim_t=0.2196, sim_o=0.2384, obj=0.1004, ||delta||=1.0000
[Iter 1] sim_t=0.2143, sim_o=0.2429, obj=0.0928, ||delta||=1.0201
[Iter 2] sim_t=0.2223, sim_o=0.2451, obj=0.0998, ||delta||=1.2203
[Iter 3] sim_t=0.2326, sim_o=0.2564, obj=0.1043, ||delta||=1.8268
[Iter 4] sim_t=0.2248, sim_o=0.2440, obj=0.1027, ||delta||=2.0216
[DEBUG] Image diff: max=0.763414, mean=0.027355


  0%|          | 0/30 [00:00<?, ?it/s]


=== Index 2 ===
Clean: A man with his shirt off sitting by the shoreline sunning hi
Target: A train station with an awning is depicted with a train on t
[CONFIG] eps_l2=10.0, step_size=1.0
[CLEAN] sim_target=0.192
[Iter 0] sim_t=0.1920, sim_o=0.2302, obj=0.0769, ||delta||=1.0000
[Iter 1] sim_t=0.1797, sim_o=0.2252, obj=0.0671, ||delta||=1.3850
[Iter 2] sim_t=0.1855, sim_o=0.2221, obj=0.0744, ||delta||=1.6636
[Iter 3] sim_t=0.1725, sim_o=0.2394, obj=0.0528, ||delta||=1.6746
[Iter 4] sim_t=0.1857, sim_o=0.2300, obj=0.0707, ||delta||=2.1235
[DEBUG] Image diff: max=0.941052, mean=0.057439


...


=== Index 10 ===
Clean: Two girls pet a goat that is in a pen.
Target: A boy is throwing a frisbee outside a building.
[CONFIG] eps_l2=10.0, step_size=1.0
[CLEAN] sim_target=0.191
[Iter 0] sim_t=0.1909, sim_o=0.1657, obj=0.1080, ||delta||=1.0000
[Iter 1] sim_t=0.1900, sim_o=0.1815, obj=0.0992, ||delta||=1.4875
[Iter 2] sim_t=0.1934, sim_o=0.1742, obj=0.1062, ||delta||=2.0102
[Iter 3] sim_t=0.1964, sim_o=0.1546, obj=0.1191, ||delta||=2.3792
[Iter 4] sim_t=0.1930, sim_o=0.1441, obj=0.1209, ||delta||=2.7958
[DEBUG] Image diff: max=0.998404, mean=0.097539


  0%|          | 0/30 [00:00<?, ?it/s]


=== Index 11 ===
Clean: A wooden teacher's desk with books stacked on it and a black
Target: A beautiful bathroom and a guy is in bathroom.
[CONFIG] eps_l2=10.0, step_size=1.0
[CLEAN] sim_target=0.148
[Iter 0] sim_t=0.1482, sim_o=0.2592, obj=0.0186, ||delta||=1.0000
[Iter 1] sim_t=0.1299, sim_o=0.2521, obj=0.0039, ||delta||=1.3231
[Iter 2] sim_t=0.1432, sim_o=0.2566, obj=0.0149, ||delta||=2.1705
[Iter 3] sim_t=0.1511, sim_o=0.2474, obj=0.0274, ||delta||=2.8219
[Iter 4] sim_t=0.2421, sim_o=0.2411, obj=0.1216, ||delta||=3.2376
[DEBUG] Image diff: max=1.000000, mean=0.168062


  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Save metrics & basic stats

In [25]:
if len(results) == 0:
    print("No results collected.")
else:
    df_res = pd.DataFrame(results)
    csv_save_path = os.path.join(OUTPUT_DIR, "final_metrics.csv")
    df_res.to_csv(csv_save_path, index=False)
    display(df_res.head())

    print("Saved metrics to:", csv_save_path)
    print("Mean clean_sim_orig :", df_res["clean_sim_orig"].mean())
    print("Mean adv_sim_orig   :", df_res["adv_sim_orig"].mean())
    print("Mean adv_sim_target :", df_res["adv_sim_target"].mean())


,index,prompt_p,target_t,clean_sim_orig,clean_sim_target,adv_sim_orig,adv_sim_target,clean_img_path,adv_img_path
0,0,a woman standing on a sidewalk while using a c...,A rode that has car and bicycle lane that cars...,0.207373,0.201866,0.207373,0.201866,/root/autodl-tmp/csc521-project/evaluation_res...,/root/autodl-tmp/csc521-project/evaluation_res...
1,1,Cat perched on the sofa looking out of the window,Stuffed toy bear sitting on dashboard of motor...,0.238351,0.219616,0.256426,0.232561,/root/autodl-tmp/csc521-project/evaluation_res...,/root/autodl-tmp/csc521-project/evaluation_res...
2,2,A man with his shirt off sitting by the shorel...,A train station with an awning is depicted wit...,0.230186,0.192014,0.230186,0.192014,/root/autodl-tmp/csc521-project/evaluation_res...,/root/autodl-tmp/csc521-project/evaluation_res...
3,3,a happy dog playing with a frisbee by a mossy ...,three birds perched and one flying towards the...,0.207700,0.213686,0.207700,0.213686,/root/autodl-tmp/csc521-project/evaluation_res...,/root/autodl-tmp/csc521-project/evaluation_res...
4,4,A road sign on top of a stop sign at an inters...,Two people with heavy winter coats are eating,0.257514,0.196445,0.257514,0.196445,/root/autodl-tmp/csc521-project/evaluation_res...,/root/autodl-tmp/csc521-project/evaluation_res...


Saved metrics to: /root/autodl-tmp/csc521-project/evaluation_results/final_metrics.csv
Mean clean_sim_orig : 0.22830773890018463
Mean adv_sim_orig   : 0.22348230670798908
Mean adv_sim_target : 0.19233283536000687


# Black Box discrete suffix search

### Helper: CLIP score for a generated image

In [ ]:
BB_OUTPUT_DIR = os.path.join(PROJECT_ROOT, "BlackBox_evaluation_results")

In [ ]:
import os
import torch
from PIL import Image

# -------------------------------------------------------------
# 1. CLIP scoring helper: CLIP(image, text)
# -------------------------------------------------------------
def clip_score_image(
    image: Image.Image,
    text: str,
    clip_model,
    clip_processor,
    device: str = "cuda",
) -> float:
    """
    Compute CLIP cosine similarity between an image and a text string.
    """
    clip_model.eval()
    with torch.no_grad():
        inputs = clip_processor(
            text=[text],
            images=[image],
            return_tensors="pt",
            padding=True,
        ).to(device)

        outputs = clip_model(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            pixel_values=inputs.pixel_values,
        )

        image_embeds = torch.nn.functional.normalize(outputs.image_embeds, dim=-1)
        text_embeds  = torch.nn.functional.normalize(outputs.text_embeds, dim=-1)

        sim = (image_embeds * text_embeds).sum(-1).item()
    return sim


# -------------------------------------------------------------
# 2. Build generic prompt candidates from clean & target captions
# -------------------------------------------------------------
def build_prompt_candidates(clean_prompt: str, target_prompt: str):
    """
    Given a clean caption and a target caption, build a small set of
    candidate adversarial prompts by combining / perturbing them.

    This is generic: it doesn't assume anything about the content.
    """
    # Basic mixing patterns
    base_patterns = [
        "{clean}",
        "{clean}. {target}",
        "{target}. {clean}",
        "{clean}, {target}",
        "{clean}. In the background, {target}",
        "A scene of {target}. {clean}",
        "A mixture of {clean} and {target}",
        "A detailed scene: {clean}. Also {target}",
    ]

    # Suffix-style patterns
    suffix_templates = [
        "{target}",
        "{target} in the background",
        "{target} in the distance",
        "featuring {target}",
        "with {target}",
        "surrounded by {target}",
        "in a scene that also shows {target}",
    ]

    candidates = []

    # Direct patterns combining clean + target
    for pattern in base_patterns:
        candidates.append(pattern.format(clean=clean_prompt, target=target_prompt))

    # Suffix patterns appended to the clean prompt
    for st in suffix_templates:
        suffix = st.format(clean=clean_prompt, target=target_prompt)
        candidates.append(f"{clean_prompt}, {suffix}")

    # Remove duplicates while preserving order
    # (dict.fromkeys trick)
    candidates = list(dict.fromkeys(candidates))

    return candidates


# -------------------------------------------------------------
# 3. Black-box prompt perturbation attack (general)
# -------------------------------------------------------------
def prompt_perturbation_attack(
    clean_prompt: str,
    target_prompt: str,
    pipe,
    clip_model,
    clip_processor,
    device: str = "cuda",
    num_inference_steps: int = 30,
    guidance_scale: float = 7.5,
    seed: int = 0,
    verbose: bool = True,
):
    """
    Black-box prompt perturbation:
    - Construct candidate prompts from clean + target captions.
    - Generate one image per candidate.
    - Choose the prompt whose image has the highest CLIP(image, target_caption).

    Returns dict with clean/adversarial prompts, images, and scores.
    """

    # 1) Generate clean image for reference
    generator = torch.Generator(device=device).manual_seed(seed)
    with torch.no_grad():
        clean_image = pipe(
            prompt=clean_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator,
        ).images[0]

    clean_score = clip_score_image(
        clean_image, target_prompt, clip_model, clip_processor, device
    )

    if verbose:
        print("-------------------------------------------------")
        print("Clean prompt: ", clean_prompt)
        print("Target prompt:", target_prompt)
        print(f"[CLEAN] CLIP(image, target) = {clean_score:.4f}")
        print("-------------------------------------------------")

    # 2) Build candidate adversarial prompts
    candidate_prompts = build_prompt_candidates(clean_prompt, target_prompt)
    if verbose:
        print(f"Total candidate prompts: {len(candidate_prompts)}")

    best_score = clean_score
    best_prompt = clean_prompt
    best_image = clean_image

    # 3) Evaluate each candidate in a black-box fashion
    for i, prompt in enumerate(candidate_prompts):
        generator = torch.Generator(device=device).manual_seed(seed)
        with torch.no_grad():
            adv_image = pipe(
                prompt=prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                generator=generator,
            ).images[0]

        score = clip_score_image(
            adv_image, target_prompt, clip_model, clip_processor, device
        )

        if verbose:
            print(f"[{i:02d}] score={score:.4f} | prompt='{prompt}'")

        if score > best_score:
            best_score = score
            best_prompt = prompt
            best_image = adv_image

    if verbose:
        print("\n================ FINAL RESULT ================")
        print("Best adversarial prompt:")
        print(best_prompt)
        print(f"Clean CLIP sim_target: {clean_score:.4f}")
        print(f"Adv   CLIP sim_target: {best_score:.4f}")
        print("=============================================\n")

    return {
        "clean_prompt": clean_prompt,
        "target_prompt": target_prompt,
        "adv_prompt": best_prompt,
        "clean_image": clean_image,
        "adv_image": best_image,
        "clean_score": clean_score,
        "adv_score": best_score,
    }

## Run on one sample pair

In [ ]:
row = df.iloc[0]
clean_prompt = row["prompt_p"]
target_prompt = row["target_t"]

result = prompt_perturbation_attack(
    clean_prompt=clean_prompt,
    target_prompt=target_prompt,
    pipe=pipe,
    clip_model=clip_model,
    clip_processor=clip_processor,
    device=device,
    num_inference_steps=30,
    guidance_scale=7.5,
    seed=0,
    verbose=True,
)



In [ ]:
# Save images
clean_path = os.path.join(BB_OUTPUT_DIR, "/bb_clean_example.png")
adv_path   = os.path.join(BB_OUTPUT_DIR, "/bb_adv_example.png")
result["clean_image"].save(clean_path)
result["adv_image"].save(adv_path)
print("Saved clean image to:", clean_path)
print("Saved adv   image to:", adv_path)

## Batch evaluation loop

In [ ]:
# import time
# import pandas as pd

# MAX_SAMPLES = 100   # e.g., 100 for debugging, or None for all

# RESULTS_CSV = os.path.join(BB_OUTPUT_DIR, "prompt_perturbation_results.csv")
# SAVE_EVERY  = 10   # save CSV every N examples
# SAVE_IMAGES = True # set False if you don't want to save example images
# IMG_SAMPLE_EVERY = 100  # save images for every N-th example


# # -------------------------------------------------------------
# # MAIN LOOP: run attack over dataset
# # -------------------------------------------------------------
# all_results = []
# num_rows = len(df) if MAX_SAMPLES is None else min(MAX_SAMPLES, len(df))
# print(f"Running prompt perturbation attack on {num_rows} examples...")

# for idx in range(num_rows):
#     row = df.iloc[idx]
#     clean_prompt = row["prompt_p"]
#     target_prompt = row["target_t"]

#     print(f"\n=== Example {idx+1}/{num_rows} ===")
#     start_time = time.time()

#     try:
#         result = prompt_perturbation_attack(
#             clean_prompt=clean_prompt,
#             target_prompt=target_prompt,
#             pipe=pipe,
#             clip_model=clip_model,
#             clip_processor=clip_processor,
#             device=device,
#             num_inference_steps=30,
#             guidance_scale=7.5,
#             seed=0,
#             verbose=False,   # per-example logs off; we print our own summary
#         )
#     except Exception as e:
#         print(f"[ERROR] Example {idx}: {e}")
#         # record failure and continue
#         all_results.append({
#             "index": idx,
#             "clean_prompt": clean_prompt,
#             "target_prompt": target_prompt,
#             "adv_prompt": None,
#             "clean_score": None,
#             "adv_score": None,
#             "improvement": None,
#             "status": f"error: {repr(e)}",
#         })
#         continue

#     elapsed = time.time() - start_time

#     clean_score = result["clean_score"]
#     adv_score   = result["adv_score"]
#     adv_prompt  = result["adv_prompt"]

#     improvement = adv_score - clean_score

#     print(f"Clean sim_target: {clean_score:.4f} | Adv sim_target: {adv_score:.4f} | Δ={improvement:.4f} | time={elapsed:.1f}s")
#     print(f"Adv prompt: {adv_prompt}")

#     # store numeric + text results
#     all_results.append({
#         "index": idx,
#         "clean_prompt": clean_prompt,
#         "target_prompt": target_prompt,
#         "adv_prompt": adv_prompt,
#         "clean_score": clean_score,
#         "adv_score": adv_score,
#         "improvement": improvement,
#         "status": "ok",
#     })

#     # optionally save some images (not all)
#     if SAVE_IMAGES and (idx % IMG_SAMPLE_EVERY == 0):
#         clean_img_path = os.path.join(BB_OUTPUT_DIR, f"bb_clean_{idx:05d}.png")
#         adv_img_path   = os.path.join(BB_OUTPUT_DIR, f"bb_adv_{idx:05d}.png")
#         result["clean_image"].save(clean_img_path)
#         result["adv_image"].save(adv_img_path)
#         print(f"Saved sample images to:\n  {clean_img_path}\n  {adv_img_path}")

#     # periodically save intermediate CSV
#     if (idx + 1) % SAVE_EVERY == 0:
#         df_results = pd.DataFrame(all_results)
#         df_results.to_csv(RESULTS_CSV, index=False)
#         print(f"[Checkpoint] Saved results for {idx+1} examples to {RESULTS_CSV}")

# # final save
# df_results = pd.DataFrame(all_results)
# df_results.to_csv(RESULTS_CSV, index=False)
# print(f"\nDone! Saved final results to {RESULTS_CSV}")
# print(df_results.describe(include='all'))


In [26]:
import os
import torch
import pandas as pd
from tqdm.notebook import tqdm
from PIL import Image

# 1. 确保有保存路径
BB_OUTPUT_DIR = os.path.join(PROJECT_ROOT, "evaluation_results", "blackbox")
os.makedirs(BB_OUTPUT_DIR, exist_ok=True)

# 2. 如果之前没运行数据加载，这里兜底加载一次 df
if 'df' not in locals():
    print("重新加载数据...")
    EXPERIMENT_CSV = os.path.join(PROJECT_ROOT, "experiment_log.csv")
    if os.path.exists(EXPERIMENT_CSV):
        df = pd.read_csv(EXPERIMENT_CSV)
    else:
        raise FileNotFoundError("找不到 experiment_log.csv，请先运行数据生成步骤！")

# 3. 定义黑盒策略 (拼接 Prompt)
def build_candidates(clean_p, target_p):
    return [
        f"{clean_p}",
        f"{clean_p} {target_p}",
        f"{target_p} {clean_p}",
        f"{clean_p}, featuring {target_p}",
        f"{clean_p} with {target_p} in the background",
        f"A mixture of {clean_p} and {target_p}",
        f"{clean_p}. {target_p}. {clean_p}",
        f"{clean_p} -- {target_p}",
    ]

# 4. CLIP 评分函数
def get_clip_score(image, text, model, processor):
    model.eval()
    model.to(device)
    with torch.no_grad():
        inputs = processor(text=[text], images=image, return_tensors="pt", padding=True).to(device)
        outputs = model(**inputs)
        img_emb = outputs.image_embeds / outputs.image_embeds.norm(dim=-1, keepdim=True)
        txt_emb = outputs.text_embeds / outputs.text_embeds.norm(dim=-1, keepdim=True)
        score = (img_emb @ txt_emb.T).item()
    return score

# 5. 开始黑盒循环
print(f"=== 开始黑盒攻击 (共 {len(df)} 条) ===")
bb_results = []

# 确保模型在 GPU 上 (如果没有跑白盒，可能需要这步)
pipe.to(device)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    clean_p = row['prompt_p']
    target_p = row['target_t']
    
    candidates = build_candidates(clean_p, target_p)
    
    best_score = -1.0
    best_image = None
    best_prompt = ""
    
    for prompt in candidates:
        # 生成图片
        with torch.no_grad():
            # 显存不够可加 pipe.enable_model_cpu_offload()
            img = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
            
        # 评分
        score = get_clip_score(img, target_p, clip_model, clip_processor)
        
        if score > best_score:
            best_score = score
            best_image = img
            best_prompt = prompt
            
    # 保存最佳结果
    save_name = f"bb_{idx}_best.png"
    best_image.save(os.path.join(BB_OUTPUT_DIR, save_name))
    
    bb_results.append({
        "index": idx,
        "clean_prompt": clean_p,
        "target_prompt": target_p,
        "best_adversarial_prompt": best_prompt,
        "best_clip_score": best_score,
        "image_path": save_name
    })

# 6. 保存数据
df_bb = pd.DataFrame(bb_results)
df_bb.to_csv(os.path.join(BB_OUTPUT_DIR, "blackbox_metrics.csv"), index=False)
print(f"黑盒攻击完成！结果已保存至: {BB_OUTPUT_DIR}")
print(f"平均 CLIP Score: {df_bb['best_clip_score'].mean():.4f}")

=== 开始黑盒攻击 (共 100 条) ===


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

